# Sanity Checks — Blink EAR Pipeline
This notebook helps you quickly verify the pipeline on **one sample video** from either the controlled or noisy folders.


**What you'll do:**

1. Configure paths and load the config.

2. Pick a sample video automatically (first found) or set one manually.

3. Run the EAR + blink detection.

4. Plot EAR vs time and print blink stats.

5. Compare short vs extended baseline segments.


In [ ]:

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from src.utils_io import load_cfg, list_videos, choose_baseline_frames
from src.blink_ear import BlinkEAR, blink_features

cfg = load_cfg("config.yaml")
print(cfg)


In [ ]:

# Try to find a sample video in controlled, else in noisy
cands = []
for d in [cfg["paths"]["controlled_dir"], cfg["paths"]["noisy_dir"]]:
    if os.path.isdir(d):
        cands.extend(list_videos(d))
if not cands:
    raise SystemExit("No videos found. Please place some videos into data/controlled or data/noisy.")
sample_video = cands[0]
sample_video


In [ ]:

# Run EAR + blink detection
be = BlinkEAR(cfg["ear_smooth_win"], cfg["blink_min_frames"], cfg["blink_merge_gap"])
ear, blinks = be.process_video(sample_video, cfg["fps_target"])
len(ear), len(blinks)


In [ ]:

# Plot EAR over time with blink intervals
t = np.arange(len(ear)) / max(cfg["fps_target"],1)
plt.figure(figsize=(10,4))
plt.plot(t, ear, linewidth=1.2)
for (s,e) in blinks:
    plt.axvspan(s/max(cfg["fps_target"],1), e/max(cfg["fps_target"],1), alpha=0.15)
plt.xlabel("Time (s)"); plt.ylabel("EAR"); plt.title("EAR with blink intervals")
plt.tight_layout()
plt.show()


In [ ]:

# Quick stats
dur_s = len(ear)/max(cfg["fps_target"],1)
rate = (len(blinks)/max(dur_s,1e-6))*60.0
print(f"Video: {sample_video}")
print(f"Duration: {dur_s:.2f}s | Blinks: {len(blinks)} | Rate: {rate:.2f} blinks/min")


In [ ]:

# Compare features in short vs extended baselines
total = len(ear)
(s1,e1),(s2,e2) = choose_baseline_frames(total, cfg["fps_target"],
                                         cfg["baseline"]["short_sec"],
                                         cfg["baseline"]["extended_sec"])

def within(seg):
    return [(max(s,seg[0]), min(e,seg[1])) for (s,e) in blinks if e>=seg[0] and s<=seg[1]]

f_short = blink_features(within((s1,e1)), cfg["fps_target"])
f_ext   = blink_features(within((s2,e2)), cfg["fps_target"])
f_short, f_ext
